In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

**一、定义对比实验的损失函数**

In [2]:
class Detect_Loss(nn.Module):

    def __init__(self, feature_size=7, num_bboxes=2, num_classes=20, lambda_coord=5.0, lambda_noobj=0.5):

        super(Detect_Loss, self).__init__()

        self.S = feature_size
        self.B = num_bboxes
        self.C = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj


    def compute_iou(self, bbox1, bbox2):

        N = bbox1.size(0)
        M = bbox2.size(0)

        lt = torch.max(
        bbox1[:, :2].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, :2].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
            )

        rb = torch.min(
        bbox1[:, 2:].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
            )

        wh = rb - lt
        wh[wh < 0] = 0
        inter = wh[:, :, 0] * wh[:, :, 1] # [N, M]

        area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1]) # [N, ]
        area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1]) # [M, ]
        area1 = area1.unsqueeze(1).expand_as(inter) # [N, ] -> [N, 1] -> [N, M]
        area2 = area2.unsqueeze(0).expand_as(inter) # [M, ] -> [1, M] -> [N, M]

        union = area1 + area2 - inter # [N, M, 2]
        iou = inter / union # [N, M, 2]

        return iou

    def forward(self, pred_tensor, target_tensor):

        device = pred_tensor.device

        S, B, C = self.S, self.B, self.C
        N = 5 * B + C

        batch_size = pred_tensor.size(0)
        coord_mask = target_tensor[:, :, :, 4] > 0
        noobj_mask = target_tensor[:, :, :, 4] == 0

        coord_mask = coord_mask.unsqueeze(-1).expand_as(target_tensor)
        noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target_tensor)

        # breakpoint()

        coord_pred = pred_tensor[coord_mask].view(-1, N)

        bbox_pred = coord_pred[:, :5 * B].contiguous().view(-1,5)
        class_pred = coord_pred[:, 5 * B:]

        coord_target = target_tensor[coord_mask].view(-1,N)

        bbox_target = coord_target[:, :5 * B].contiguous().view(-1, 5)
        class_target = coord_target[:, 5 * B:]

        noobj_pred = pred_tensor[noobj_mask].view(-1,N)

        noobj_target = target_tensor[noobj_mask].view(-1,N)

        noobj_conf_mask = torch.cuda.BoolTensor(noobj_pred.size()).fill_(0).to(device)
        for b in range(B):
            noobj_conf_mask[:, 4 + b * 5] = 1
        noobj_pred_conf = noobj_pred[noobj_conf_mask]
        noobj_target_conf = noobj_target[noobj_conf_mask]
        loss_noobj = F.mse_loss(noobj_pred_conf, noobj_target_conf, reduction='sum')

        coord_response_mask = torch.cuda.BoolTensor(bbox_target.size()).fill_(0).to(device)
        coord_not_response_mask = torch.cuda.BoolTensor(bbox_target.size()).fill_(1).to(device)
        bbox_target_iou = torch.zeros(bbox_target.size()).to(device)

        for i in range(0, bbox_target.size(0), B):
            pred = bbox_pred[i:i + B]
            pred_xyxy = Variable(torch.FloatTensor(pred.size()))

            pred_xyxy[:, :2] = pred[:, :2] / float(S) - 0.5 * pred[:, 2:4]
            pred_xyxy[:, 2:4] = pred[:, :2] / float(S) + 0.5 * pred[:, 2:4]

            target = bbox_target[i].view(-1, 5)
            target_xyxy = Variable(torch.FloatTensor(target.size()))

            target_xyxy[:, :2] = target[:, :2] / float(S) - 0.5 * target[:, 2:4]
            target_xyxy[:, 2:4] = target[:, :2] / float(S) + 0.5 * target[:, 2:4]

            iou = self.compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4])
            max_iou, max_index = iou.max(0)
            max_index = max_index.data

            coord_response_mask[i + max_index] = 1
            coord_not_response_mask[i+max_index] = 0

            bbox_target_iou[i + max_index, torch.LongTensor([4])] = (max_iou).data.to(device)
        bbox_target_iou = Variable(bbox_target_iou).to(device)

        bbox_pred_response = bbox_pred[coord_response_mask].view(-1, 5)
        bbox_target_response = bbox_target[coord_response_mask].view(-1,5)
        target_iou = bbox_target_iou[coord_response_mask].view(-1,5)
        loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
        loss_wh = F.mse_loss(torch.sqrt(bbox_pred_response[:, 2:4]), torch.sqrt(bbox_target_response[:, 2:4]),reduction='sum')
        loss_obj = F.mse_loss(bbox_pred_response[:, 4], target_iou[:, 4], reduction='sum')

        loss_class = F.mse_loss(class_pred, class_target, reduction='sum')

        # breakpoint()

        loss = self.lambda_coord * (loss_xy + loss_wh) + loss_obj + self.lambda_noobj * loss_noobj + loss_class
        loss = loss / float(batch_size)

        return loss


In [3]:
class YoloLoss(nn.Module):
    """定义一个为yolov1的损失函数"""

    def __init__(self,feature_size=7, num_bboxes=2, num_classes=20, lambda_coord=5.0, lambda_noobj=0.5):
        super().__init__()
        self.S = feature_size
        self.B = num_bboxes
        self.C = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def compute_iou(self,bbox1,bbox2):
        """"
        计算两组边界框之间的交并比(IoU)
        
        参数:
        - bbox1: 形状为 [N, 4] 的边界框 (xmin, ymin, xmax, ymax)
        - bbox2: 形状为 [M, 4] 的边界框 (xmin, ymin, xmax, ymax)
        
        返回:
        - iou: 形状为 [N, M] 的 IoU 矩阵
        """
        # 获取边界框数量
        N = bbox1.size(0)
        M = bbox2.size(0)
        
        # 计算交集的左上角坐标 (left-top)
        lt = torch.max(
            bbox1[:, :2].unsqueeze(1).expand(N, M, 2),  # [N, 2] -> [N, 1, 2] -> [N, M, 2]
            bbox2[:, :2].unsqueeze(0).expand(N, M, 2)   # [M, 2] -> [1, M, 2] -> [N, M, 2]
        )
        
        # 计算交集的右下角坐标 (right-bottom)
        rb = torch.min(
            bbox1[:, 2:].unsqueeze(1).expand(N, M, 2),  # [N, 2] -> [N, 1, 2] -> [N, M, 2]
            bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)    # [M, 2] -> [1, M, 2] -> [N, M, 2]
        )
        
        # 计算交集的宽高
        wh = rb - lt
        wh[wh < 0] = 0  # 处理无重叠的情况
        inter = wh[:, :, 0] * wh[:, :, 1]  # 交集面积 [N, M]
        
        # 计算两个边界框各自的面积
        area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1])  # [N, ]
        area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1])  # [M, ]
        area1 = area1.unsqueeze(1).expand_as(inter)  # [N, ] -> [N, 1] -> [N, M]
        area2 = area2.unsqueeze(0).expand_as(inter)  # [M, ] -> [1, M] -> [N, M]
        
        # 计算并集面积
        union = area1 + area2 - inter  # [N, M]
        
        # 计算 IoU
        iou = inter / union  # [N, M]
        
        return iou


    def forward(self,pred:torch.Tensor,target:torch.Tensor):
        """
        计算 YOLOv1 损失
        
        参数:
        - pred_tensor: 模型预测的输出张量，形状为 [batch_size, S, S, B*5 + C]
        - target_tensor: 目标标签张量，形状与 pred_tensor 相同
        
        返回:
        - loss: 计算得到的损失值
        """
        # target/pred = (N,C,H,W) -> (N,H,W,C)
        device = target.device
        target = target.permute(0,2,3,1)
        pred = pred.permute(0,2,3,1)
        batch_size = pred.shape[0]

        # breakpoint()

        # 设置临时参数，减少重复self引用
        S = self.S
        B = self.B
        C = self.C
        grid_size = 1.0 / S # 归一化的网格大小

        # 设置有目标的mask和没目标的mask
        coord_mask = target[:,:,:,4] > 0
        noobj_mask = target[:,:,:,4] == 0
        coord_mask = coord_mask.unsqueeze(-1).expand_as(target)
        noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target)

        # 提取有目标的pred和没目标的pred
        coord_pred = pred[coord_mask].reshape(-1, 5 * B + C)
        noobj_pred = pred[noobj_mask].reshape(-1, 5 * B + C)

        # 提取有目标的target和没目标的target
        coord_target = target[coord_mask].reshape(-1, 5 * B + C)
        noobj_target = target[noobj_mask].reshape(-1, 5 * B + C)

        # 提取bbox与class
        bbox_pred = coord_pred[:,:5 * B].reshape(-1, 5)
        class_pred = coord_pred[:,5 * B:]
        bbox_target = coord_target[:,:5 * B].reshape(-1, 5)
        class_target = coord_target[:,5 * B:]

        # 处理无目标位置的置信度损失
        noobj_conf_mask = torch.BoolTensor(noobj_pred.shape).fill_(0).to(device)
        for b in range(B):
            noobj_conf_mask[:,4 + b * 5] = 1 # 设置提取出置信度的位置
        noobj_conf_pred = noobj_pred[noobj_conf_mask]
        noonj_conf_target = noobj_target[noobj_conf_mask]

        # 计算noobj_loss_conf
        loss_conf_noobj = F.mse_loss(noobj_conf_pred,noonj_conf_target,reduction = 'sum')

        # 初始化响应掩码
        coord_response_mask = torch.BoolTensor(bbox_target.size()).fill_(0).to(device) # 响应初始化为0
        coord_not_response_mask = torch.BoolTensor(bbox_target.size()).fill_(1).to(device) # 非响应初始化为1
        bbox_target_iou = torch.zeros(bbox_target.size()).to(device)

        # 遍历每个目标网格
        for i in range(0,bbox_pred.shape[0],B):
            # 获取当前网格的 B 个预测边界框
            pred = bbox_pred[i:i + B] 

            # 将预测边界框转换为 (xmin, ymin, xmax, ymax) 格式
            pred_xyxy = torch.zeros((pred.shape[0],4)).to(device)
            pred_xyxy[:,:2] = pred[:,:2] * grid_size - pred[:,2:4] * 0.5    # 左上
            pred_xyxy[:,2:4] = pred[:,:2] * grid_size + pred[:,2:4] * 0.5   # 右下

            # 同样处理出目标坐标
            target = bbox_target[i].reshape(-1,5)
            target_xyxy = torch.zeros((target.shape[0],4)).to(device)
            target_xyxy[:,:2] = target[:,:2] * grid_size - target[:,2:4] * 0.5    # 左上
            target_xyxy[:,2:4] = target[:,:2] * grid_size + target[:,2:4] * 0.5   # 右下

            # 计算iou
            iou = self.compute_iou(pred_xyxy,target_xyxy)
            max_iou, max_index = iou.max(0)  # 找出最大 IoU 及其索引

            # 标记负责预测目标的边界框
            coord_response_mask[i + max_index] = 1
            coord_not_response_mask[i + max_index] = 0

            # 将最大 IoU 作为置信度目标
            bbox_target_iou[i + max_index,4] = max_iou.data

        # 提取负责预测目标的边界框
        bbox_pred_response = bbox_pred[coord_response_mask].reshape(-1, 5)
        bbox_target_response = bbox_target[coord_response_mask].reshape(-1, 5)
        target_iou = bbox_target_iou[coord_response_mask].reshape(-1, 5)

        # 计算坐标损失 (中心点坐标)
        loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
        
        # 计算宽高损失
        loss_wh = F.mse_loss(
            torch.sqrt(bbox_pred_response[:,2:4]), 
            torch.sqrt(bbox_target_response[:,2:4]), 
            reduction = 'sum'
        )

        # 计算目标置信度损失
        loss_conf = F.mse_loss(bbox_pred_response[:,4], target_iou[:,4], reduction = 'sum')

        # 计算类别预测损失
        loss_class = F.mse_loss(class_pred,class_target,reduction = 'sum')

        # breakpoint()

        loss = (self.lambda_coord * (loss_xy + loss_wh) + 
                self.lambda_noobj * loss_conf_noobj +
                loss_conf + loss_class)
        
        return loss / batch_size # 平均损失

In [4]:
loss = YoloLoss()
loss_re = Detect_Loss()

**二、使用模拟数据进行测试**

In [5]:
x = torch.zeros((2,2))
mask = torch.BoolTensor([[1,1],[0,0]])
x[mask] = 1
print(x)

tensor([[1., 1.],
        [0., 0.]])


In [6]:
# 注意要符合数据的要求，归一化后数值0~1
pre = torch.rand(1,30,7,7)  # 随机预测张量 [batch, S, S, B*5+C]
tar = torch.rand(1,30,7,7)  # 随机目标张量
mask1 = pre > 0
mask2 = tar > 0
pre[~mask1] = 0
tar[~mask2] = 0
print(pre)

tensor([[[[0.3875, 0.0667, 0.8786,  ..., 0.9227, 0.6411, 0.3652],
          [0.7810, 0.1526, 0.8879,  ..., 0.4373, 0.3374, 0.6314],
          [0.3279, 0.7227, 0.7934,  ..., 0.6968, 0.5672, 0.5699],
          ...,
          [0.4301, 0.5339, 0.0453,  ..., 0.2022, 0.2803, 0.5357],
          [0.1765, 0.6764, 0.2145,  ..., 0.3495, 0.6277, 0.4965],
          [0.6739, 0.1320, 0.4863,  ..., 0.3956, 0.4727, 0.4734]],

         [[0.8189, 0.8120, 0.6477,  ..., 0.3683, 0.9540, 0.0100],
          [0.2414, 0.0621, 0.7469,  ..., 0.2494, 0.3464, 0.1221],
          [0.4569, 0.1222, 0.4193,  ..., 0.4081, 0.1499, 0.6103],
          ...,
          [0.4739, 0.0052, 0.8733,  ..., 0.2954, 0.2810, 0.5989],
          [0.7181, 0.3405, 0.3143,  ..., 0.6827, 0.4679, 0.6387],
          [0.1674, 0.2090, 0.5196,  ..., 0.2532, 0.9658, 0.0382]],

         [[0.7914, 0.4903, 0.8307,  ..., 0.8920, 0.5298, 0.9164],
          [0.4516, 0.7117, 0.7247,  ..., 0.2350, 0.0833, 0.8645],
          [0.2048, 0.3564, 0.5436,  ..., 0

In [7]:
out1 = loss(pre,tar)

In [8]:
out2 = loss_re(pre.permute(0,2,3,1),tar.permute(0,2,3,1))

/tmp/ipykernel_2089225/2646532065.py:73: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  noobj_conf_mask = torch.cuda.BoolTensor(noobj_pred.size()).fill_(0).to(device)


In [9]:
print(out1)
print(out2)

tensor(295.1829)
tensor(295.1829)


**损失函数正常工作，无Nan**

**三、使用构建的数据集进行测试**

In [10]:
from utils.datasets import YoloData
import torchvision
transforms = torchvision.transforms.Compose(
        [torchvision.transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std  = [0.229, 0.224, 0.225]
        ),torchvision.transforms.Resize((448,448))]
    )

# 加载数据集
yolodata = YoloData("datasets/JPEGImages","datasets/train.txt",transforms = transforms)

Generating targets: 100%|██████████| 13700/13700 [00:03<00:00, 4370.18it/s]


In [13]:
from torch.utils import data
batch_size = 1
train_iter = data.DataLoader(yolodata,batch_size,shuffle = True)

In [14]:
for batch in train_iter:
    X,Y = batch
    break

In [15]:
print(X.shape,Y.shape)

torch.Size([1, 3, 448, 448]) torch.Size([1, 30, 7, 7])


In [11]:
from modules.yolov1 import Yolov1
net = Yolov1()

In [16]:
y_pred = net(X)
# mask = y_pred < 0
# y_pred[mask] = 0
# print(y_pred.shape)

In [17]:
out1 = loss(y_pred,Y)
out2 = loss_re(y_pred.permute(0,2,3,1),Y.permute(0,2,3,1))

tensor(0.2949, grad_fn=<MseLossBackward0>) tensor(0.7297, grad_fn=<MseLossBackward0>) tensor(0.9243, grad_fn=<MseLossBackward0>) tensor(2.8729e-05, grad_fn=<MseLossBackward0>) tensor(0.2928, grad_fn=<MseLossBackward0>)
tensor([0.0054])
area1:tensor([[0.0000],
        [0.0025]], grad_fn=<ExpandBackward0>)
area2:tensor([[0.4715],
        [0.4715]])
inter:tensor([[0.0000],
        [0.0025]], grad_fn=<MulBackward0>)
union:tensor([[0.4715],
        [0.4715]], grad_fn=<SubBackward0>)
tensor(0.2949, grad_fn=<MseLossBackward0>) tensor(0.7297, grad_fn=<MseLossBackward0>) tensor(0.9243, grad_fn=<MseLossBackward0>) tensor(2.8729e-05, grad_fn=<MseLossBackward0>) tensor(0.2928, grad_fn=<MseLossBackward0>)
tensor([0.0054])


In [18]:
print(out1,out2)

tensor(6.1938, grad_fn=<DivBackward0>) tensor(6.1938, grad_fn=<DivBackward0>)


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Detect_Loss_raw(nn.Module):

    def __init__(self, feature_size=7, num_bboxes=2, num_classes=20, lambda_coord=5.0, lambda_noobj=0.5):

        super().__init__()

        self.S = feature_size
        self.B = num_bboxes
        self.C = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj


    def compute_iou(self, bbox1, bbox2):

        N = bbox1.size(0)
        M = bbox2.size(0)

        lt = torch.max(
        bbox1[:, :2].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, :2].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
            )

        rb = torch.min(
        bbox1[:, 2:].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
        bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
            )

        wh = rb - lt
        wh[wh < 0] = 0
        inter = wh[:, :, 0] * wh[:, :, 1] # [N, M]

        area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1]) # [N, ]
        area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1]) # [M, ]
        area1 = area1.unsqueeze(1).expand_as(inter) # [N, ] -> [N, 1] -> [N, M]
        area2 = area2.unsqueeze(0).expand_as(inter) # [M, ] -> [1, M] -> [N, M]

        union = area1 + area2 - inter # [N, M, 2]
        iou = inter / union # [N, M, 2]

        return iou

    def forward(self, pred_tensor, target_tensor):

        S, B, C = self.S, self.B, self.C
        N = 5 * B + C

        batch_size = pred_tensor.size(0)
        coord_mask = target_tensor[:, :, :, 4] > 0
        noobj_mask = target_tensor[:, :, :, 4] == 0

        coord_mask = coord_mask.unsqueeze(-1).expand_as(target_tensor)
        noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target_tensor)

        coord_pred = pred_tensor[coord_mask].view(-1, N)

        bbox_pred = coord_pred[:, :5 * B].contiguous().view(-1,5)
        class_pred = coord_pred[:, 5 * B:]

        coord_target = target_tensor[coord_mask].view(-1,N)

        bbox_target = coord_target[:, :5 * B].contiguous().view(-1, 5)
        class_target = coord_target[:, 5 * B:]

        noobj_pred = pred_tensor[noobj_mask].view(-1,N)

        noobj_target = target_tensor[noobj_mask].view(-1,N)

        noobj_conf_mask = torch.cuda.BoolTensor(noobj_pred.size()).fill_(0)
        for b in range(B):
            noobj_conf_mask[:, 4 + b * 5] = 1
        noobj_pred_conf = noobj_pred[noobj_conf_mask]
        noobj_target_conf = noobj_target[noobj_conf_mask]
        loss_noobj = F.mse_loss(noobj_pred_conf, noobj_target_conf, reduction='sum')

        coord_response_mask = torch.cuda.BoolTensor(bbox_target.size()).fill_(0)
        coord_not_response_mask = torch.cuda.BoolTensor(bbox_target.size()).fill_(1)
        bbox_target_iou = torch.zeros(bbox_target.size()).cuda()

        for i in range(0, bbox_target.size(0), B):
            pred = bbox_pred[i:i + B]
            pred_xyxy = Variable(torch.FloatTensor(pred.size()))

            pred_xyxy[:, :2] = pred[:, :2] / float(S) - 0.5 * pred[:, 2:4]
            pred_xyxy[:, 2:4] = pred[:, :2] / float(S) + 0.5 * pred[:, 2:4]

            target = bbox_target[i].view(-1, 5)
            target_xyxy = Variable(torch.FloatTensor(target.size()))

            target_xyxy[:, :2] = target[:, :2] / float(S) - 0.5 * target[:, 2:4]
            target_xyxy[:, 2:4] = target[:, :2] / float(S) + 0.5 * target[:, 2:4]

            iou = self.compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4])
            max_iou, max_index = iou.max(0)
            max_index = max_index.data.cuda()

            coord_response_mask[i + max_index] = 1
            coord_not_response_mask[i+max_index] = 0

            bbox_target_iou[i + max_index, torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
        bbox_target_iou = Variable(bbox_target_iou).cuda()

        bbox_pred_response = bbox_pred[coord_response_mask].view(-1, 5)
        bbox_target_response = bbox_target[coord_response_mask].view(-1,5)
        target_iou = bbox_target_iou[coord_response_mask].view(-1,5)
        loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
        loss_wh = F.mse_loss(torch.sqrt(bbox_pred_response[:, 2:4]), torch.sqrt(bbox_target_response[:, 2:4]),reduction='sum')
        loss_obj = F.mse_loss(bbox_pred_response[:, 4], target_iou[:, 4], reduction='sum')

        loss_class = F.mse_loss(class_pred, class_target, reduction='sum')

        print(loss_xy,loss_wh,loss_class,loss_obj,loss_noobj)

        loss = self.lambda_coord * (loss_xy + loss_wh) + loss_obj + self.lambda_noobj * loss_noobj + loss_class
        loss = loss / float(batch_size)

        return loss

In [20]:
loss2 = Detect_Loss_raw()
device = torch.device("cuda:0")
out3 = loss2(y_pred.permute(0,2,3,1).to(device),Y.permute(0,2,3,1).to(device))

tensor(0.2949, device='cuda:0', grad_fn=<MseLossBackward0>) tensor(0.7297, device='cuda:0', grad_fn=<MseLossBackward0>) tensor(0.9243, device='cuda:0', grad_fn=<MseLossBackward0>) tensor(2.8729e-05, device='cuda:0', grad_fn=<MseLossBackward0>) tensor(0.2928, device='cuda:0', grad_fn=<MseLossBackward0>)


/tmp/ipykernel_1771900/926102265.py:74: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  noobj_conf_mask = torch.cuda.BoolTensor(noobj_pred.size()).fill_(0)


In [21]:
print(out3)

tensor(6.1938, device='cuda:0', grad_fn=<DivBackward0>)


---

In [3]:
ddd = torch.randn((2,2))
print(ddd)
print(torch.sqrt(ddd))

tensor([[ 0.0927,  1.1068],
        [ 0.3758, -0.8797]])
tensor([[0.3045, 1.0521],
        [0.6130,    nan]])
